loading iris data and splitting the data as X_train,X_test,y_train,y_test

In [47]:
import numpy as np
import math
from sklearn.datasets import load_iris
iris = load_iris()
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(iris['data'],iris['target'],random_state=142)



loading ionsphere data and splitting the data as X_train1,X_test1,y_train1,y_test1

In [48]:
X = np.genfromtxt("ionosphere[1].txt", delimiter=",",usecols = np.arange(34))
Y = np.genfromtxt("ionosphere[1].txt", delimiter=",",usecols = 34)
from sklearn.model_selection import train_test_split
X_train1,X_test1,y_train1,y_test1 = train_test_split(X,Y,random_state=142)


Recreating conformal prediction using the examples in the slide

In [63]:
X_train3 =np.array([[0,3],[2,2],[3,3],[-1,1],[-1,-1], [0,1]])
y_train3 =np.array([1,1,1,0,0,0])
X_test3 =np.array([[0,0]])

# Algorithm for nearest neighbour

In [50]:
def min(x):
    
    minimum = math.inf               #method for finding shortest distance
    for i in x:
        if i < minimum:
            minimum = i
    return minimum  

In [51]:

def NN(X_test,X_train,y_train,y_test):
    
    truelabels = 0
    falselabels = 0
    distance = np.zeros(len(X_train))
    label = []
    
    for X in X_test:                                         
        for i in range(len(X_train)):
            sumsquares = 0
            for j in range(len(X)):
                sumsquares += (X[j] - X_train[i][j])**2
            distance[i] = math.sqrt(sumsquares)

            
        nearest_distance = min(distance)         #using min method to obtain shortest distance
        index = np.argmin(distance)
        nearest_label = y_train[index]
        label.append(nearest_label)         #storing the labels of nearest neighbours in the variable 'label'
        
        
    for k in range (len(X_test)):
        a = (bool(label[k] == y_test[k]))   #checking if the predicted label is equal to true label
        if (a == True):
            truelabels += 1
        else:                          
            falselabels +=1              
            
    
    error_rate = falselabels/len(X_test)
    success_rate = truelabels/len(X_test)
    print(error_rate)
    
 

Error rate for iris dataset

In [54]:
NN(X_test,X_train,y_train,y_test)

0.10526315789473684


Error rate for ionsphere dataset

In [53]:
NN(X_test1,X_train1,y_train1,y_test1)

0.19318181818181818


# conformal prediction

"method to find the distance between two arrays"

In [55]:
# 
def eucledian(a,b):
        import math
        sumsquares = 0
        for i in range (len (a)):
            
            sumsquares += ((a[i]-b[i])**2)
            distance =math.sqrt(sumsquares)
        return(distance)
        


"method to find the rank" 

In [56]:
# a = list of alphas = list of conformity score
def ranking(a):
    test_score = a[-1]
    rank=0
    for i in range(len(a)):
        if test_score >= a[i]:
            rank += 1
    return rank


"method to do division"

In [57]:
# a = distance to nearest sample of different class
# b = distance to nearest sample of same class

def division(a,b):
    if b == 0:
        return math.inf
    else:
        return a/b
    


method to find the confirmity score and  pvalue

In [58]:
def conformity(extendedtrainingset,extendedlabels):
    

    confirmity_measures = []
    
    for i,sample1 in enumerate (extendedtrainingset):
        min_dist_same_class = math.inf
        min_dist_diff_class = math.inf
        confirmity_measure = 0
        
        for j,sample2 in enumerate (extendedtrainingset):  
            if (j != i):                                          #(To make sure we are not calculating distance with the same sample)
                
                if extendedlabels[i] == extendedlabels[j]:        #(for selecting samples from same class)
                    
                
                    dist_same_class = eucledian(sample1,sample2)
                    if dist_same_class < min_dist_same_class:
                        min_dist_same_class = dist_same_class
                    
                else:                                                   
                    dist_diff_class = eucledian(sample1,sample2)
                    if dist_diff_class < min_dist_diff_class:
                        min_dist_diff_class = dist_diff_class
        confirmity_measure = division(min_dist_diff_class,min_dist_same_class) #(using division function to find confirmity score)
        confirmity_measures.append(confirmity_measure)
        
    rank = ranking(confirmity_measures)                      #(using ranking function to find the rank
                                                            # and then finding the p values)
    p_value = rank/len(extendedtrainingset)
    return (p_value)
    
    
    

"method to iterate over test samples and concatenate each with the training samples and calling methods"

In [59]:

 
def conformalprediction(X_train,X_test,y_train):
    p_values_of_xtests=[]
    p_value_of_xtests = []
    sum = 0 
    labels = np.unique(y_train)
    for i in range (len(X_test)):
        a = np.array([X_test[i]])
        extendedtrainingset = np.concatenate((X_train,a))        #(making extendedtrainingset by combaining training set with test set) ) 
        p_values_of_xtest=[]
        for j in range(len(labels)):
            samplespace = np.array([labels[j]])
            extendedlabels = np.concatenate((y_train,samplespace))   #(combaining all possible labels of test set with labels of training set)
            
            p_value_of_xtest = conformity(extendedtrainingset,extendedlabels)
            p_values_of_xtest.append(p_value_of_xtest)
        
        p_values_of_xtests.append(p_values_of_xtest)
        
    return(p_values_of_xtests)         
       



P_values of iris dataset

In [60]:
print(conformalprediction(X_train,X_test,y_train))


[[0.8407079646017699, 0.008849557522123894, 0.008849557522123894], [0.008849557522123894, 0.5752212389380531, 0.008849557522123894], [0.008849557522123894, 0.08849557522123894, 0.008849557522123894], [0.008849557522123894, 0.008849557522123894, 0.6548672566371682], [0.008849557522123894, 0.584070796460177, 0.008849557522123894], [0.008849557522123894, 0.18584070796460178, 0.008849557522123894], [0.9911504424778761, 0.008849557522123894, 0.008849557522123894], [0.8761061946902655, 0.008849557522123894, 0.008849557522123894], [0.008849557522123894, 0.008849557522123894, 0.168141592920354], [0.008849557522123894, 0.20353982300884957, 0.008849557522123894], [0.008849557522123894, 0.2743362831858407, 0.008849557522123894], [0.008849557522123894, 0.584070796460177, 0.008849557522123894], [0.008849557522123894, 0.008849557522123894, 0.5486725663716814], [1.0, 0.008849557522123894, 0.008849557522123894], [0.008849557522123894, 0.017699115044247787, 0.04424778761061947], [0.8053097345132744, 0.

P_values of Ionsphere Dataset

In [62]:
print(conformalprediction(X_train1,X_test1,y_train1))

[[0.2840909090909091, 0.041666666666666664], [0.003787878787878788, 0.821969696969697], [0.003787878787878788, 0.6893939393939394], [0.003787878787878788, 0.821969696969697], [0.030303030303030304, 0.3333333333333333], [0.2159090909090909, 0.06818181818181818], [0.10984848484848485, 0.13257575757575757], [0.01893939393939394, 0.3522727272727273], [0.003787878787878788, 0.5454545454545454], [0.003787878787878788, 0.8863636363636364], [0.003787878787878788, 0.7840909090909091], [0.056818181818181816, 0.2537878787878788], [0.08333333333333333, 0.17424242424242425], [0.015151515151515152, 0.36363636363636365], [0.11742424242424243, 0.11742424242424243], [0.011363636363636364, 0.38257575757575757], [0.17045454545454544, 0.08333333333333333], [0.003787878787878788, 0.6174242424242424], [0.003787878787878788, 0.6553030303030303], [0.003787878787878788, 0.8371212121212122], [0.003787878787878788, 0.8522727272727273], [0.003787878787878788, 0.7083333333333334], [0.3143939393939394, 0.0416666666

P_values of the samples in the slide

In [61]:
print(conformalprediction(X_train3,X_test3,y_train3))

[[0.8571428571428571, 0.14285714285714285]]
